<a href="https://colab.research.google.com/github/andersonnb6/ColabPython-Tutoriais/blob/main/Manipulando_GSheets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Manipulando GSheets com Google Colab (Python)**

## Preparando ambiente
Ao executar esta etapa será gerado um link para autenticação. O link abrirá uma página com um código de autendicação que deve ser inserido na saída do comando.

In [1]:
# Atualizar pacote necessário para ler .gsheets
%pip install --upgrade gspread

# Autenticar acesso na sua conta google para acessar arquivos
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

## Carregando e Manipulando GSheets

In [2]:
# Carregando dados
planilha = gc.open('db_mtcars')

In [3]:
# Especificando o uso da primeira aba da planilha
df_aba1 = planilha.sheet1

In [4]:
# Mostrando primeira linha
df_aba1.row_values(1)

['model',
 'mpg',
 'cyl',
 'disp',
 'hp',
 'drat',
 'wt',
 'qsec',
 'vs',
 'am',
 'gear',
 'carb']

In [5]:
# Mostrando valor de célula específica (Modo 1)
df_aba1.acell('A2').value

'Mazda RX4'

In [6]:
# Mostrando valor de célula específica (Modo 2)
df_aba1.cell(2,1).value # (linha, coluna)

'Mazda RX4'

In [7]:
# Atualizar valor de uma célula
df_aba1.update('M1','Testando')

{'spreadsheetId': '1vVJNV9g1I6L9piR8iSG0OlgpHwle8pkBL0i8D-nX_Ok',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': "'Página1'!M1",
 'updatedRows': 1}

In [8]:
# Apagar valor de uma célula
df_aba1.update('M1','')

{'spreadsheetId': '1vVJNV9g1I6L9piR8iSG0OlgpHwle8pkBL0i8D-nX_Ok',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': "'Página1'!M1",
 'updatedRows': 1}

## Convertendo GSheets em datraframe Pandas

In [9]:
# Importando Pandas
import pandas as pd

In [10]:
# Convertendo em dataframe Pandas
df_aba1_pandas = pd.DataFrame(df_aba1.get_all_records())

In [11]:
# Mostrando dataframe no formato Pandas
df_aba1_pandas.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


## Exportando do Pandas para GSheets

In [12]:
# Filtrando dados mpg > 30
df_aba1_pandas_filtered = df_aba1_pandas.query('mpg>30')
df_aba1_pandas_filtered.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
17,Fiat 128,32.4,4,78.7,66,4.08,2.200,19.47,1,1,4,1
18,Honda Civic,30.4,4,75.7,52,4.93,1.615,18.52,1,1,4,2
19,Toyota Corolla,33.9,4,71.1,65,4.22,1.835,19.90,1,1,4,1
27,Lotus Europa,30.4,4,95.1,113,3.77,1.513,16.90,1,1,5,2


In [13]:
# Descobrindo tamanho do dataframe filtrado (4x12)
df_aba1_pandas_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 17 to 27
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   model   4 non-null      object 
 1   mpg     4 non-null      float64
 2   cyl     4 non-null      int64  
 3   disp    4 non-null      float64
 4   hp      4 non-null      int64  
 5   drat    4 non-null      float64
 6   wt      4 non-null      float64
 7   qsec    4 non-null      float64
 8   vs      4 non-null      int64  
 9   am      4 non-null      int64  
 10  gear    4 non-null      int64  
 11  carb    4 non-null      int64  
dtypes: float64(5), int64(6), object(1)
memory usage: 416.0+ bytes


In [14]:
# Criando nova aba no .gsheets para inserir dados filtrados
# Se já houver uma aba com o mesmo nome dará erro
new_aba = planilha.add_worksheet(title="Nova_Aba", rows="4", cols="12")

In [15]:
# Enviando dados filtrados para nova aba do gsheets
new_aba.update([df_aba1_pandas_filtered.columns.values.tolist()] + df_aba1_pandas_filtered.values.tolist())

{'spreadsheetId': '1vVJNV9g1I6L9piR8iSG0OlgpHwle8pkBL0i8D-nX_Ok',
 'updatedCells': 60,
 'updatedColumns': 12,
 'updatedRange': 'Nova_Aba!A1:L5',
 'updatedRows': 5}